In [6]:
using PyCall
using Statistics
using LinearAlgebra
using Random
using Distributions
using TensorCrossInterpolation
import TensorCrossInterpolation as TCI
using JLD2
using Tensor_FixedSeedMC
import Tensor_FixedSeedMC as TCIMC

np = pyimport("numpy")
@pyimport tf_quant_finance as tff
@pyimport tensorflow as tf


In [7]:
# 満期・金利・行使価格など
T = 1.0               # 満期
r = 0.01              # 無リスク金利
K = 100.0             # 行使価格
d = 5                 # 原資産数
nPath = 10^1           # モンテカルロパス数
S0 = fill(100, d)     # 原資産の初期値ベクトル
seed = 1234

function chebyshev_lobatto_nodes(n::Int, a::Real, b::Real)
    # [-1,1] のチェビシェフ-ロバット節点 → [a,b] へ写像
    x_standard = [cos(π * (n - 1 - i) / (n - 1)) for i in 0:n-1]
    x_scaled = 0.5 * (b - a) .* (x_standard .+ 1) .+ a
    return x_scaled, x_standard
end

num_nodes = 100
σs, _ = chebyshev_lobatto_nodes(num_nodes, 0.15, 0.25)    # ボラティリティグリッド
stock0, _ = chebyshev_lobatto_nodes(num_nodes, 90, 120)   # 初期株価グリッド

corrMat = [
    1.0      0.303659  0.505532  0.719655  0.728832;
    0.303659 1.0      0.401077  0.515272  0.178203;
    0.505532 0.401077 1.0      0.132542  0.722767;
    0.719655 0.515272 0.132542 1.0      0.394723;
    0.728832 0.178203 0.722767 0.394723  1.0
]

function calculate_option_price(
    T::Float64, r::Float64, K::Float64, d::Int, S0::Vector{Int}, stock0::Vector{Float64},
    localindex_v::Vector{Int}, nPath::Int, σs::Vector{Float64}, corrMat
)::Tuple{Float64, Float64}
    # ボラリティ、初期株価の組み合わせを抽出
    vols = [σs[i] for i in localindex_v[1:2:2*d]]
    S0s = [stock0[i] for i in localindex_v[2:2:2*d]]

    np.random.seed(seed)  # numpyのシードを固定
    tf.random.set_seed(seed)  # tensorflowのシードを固定
    
    process = tff.models.MultivariateGeometricBrownianMotion(
        dim=d,
        means=fill(r, d),
        volatilities=vols,
        corr_matrix=corrMat)
    
    paths = process.sample_paths(
        times=[T],
        initial_state=S0s,
        random_type=tff.math.random.RandomType.PSEUDO,
        seed = seed,
        num_samples=nPath
    ).numpy()
    
    payoffs = max.(minimum(paths[:, 1, :], dims=2) .- K, 0.0)
    mean_pv = exp(-r * T) * mean(payoffs)
    std_pv = exp(-r * T) * std(payoffs)
    return (mean_pv, std_pv)
end


calculate_option_price (generic function with 1 method)

In [8]:
# struct to count number of function calls and sampling points
mutable struct WrapFuncFloat <: Function
    f::Function
    #count::Int64
    itpl_point_dict::Dict{Vector{Int}, Float64}
    function WrapFuncFloat(f::Function)::WrapFuncFloat
        new(f, Dict{Vector{Int}, Float64}())
    end
end



function (w::WrapFuncFloat)(bins::Vector{Int})::Float64
    if !haskey(w.itpl_point_dict, bins)
        # FIXED: use deepcopy
        w.itpl_point_dict[deepcopy(bins)] = w.f(bins)
    end
    return w.itpl_point_dict[bins]
end
;

function mps2multivec(mps::TCI.TensorTrain)::Vector{Array{Float64, 3}}
    links = linkinds(mps)
    sites = siteinds(mps)
    N = length(mps)
    res = []
    for i in 1:N
        if i == 1
            push!(res, reshape(array(mps[1], sites[1], links[1]), 1, ITensors.dim(sites[1]), ITensors.dim(links[1])))
        elseif i == N
            push!(res, reshape(array(mps[i], links[end], sites[i]), ITensors.dim(links[end]), ITensors.dim(sites[i]), 1))
        else
            push!(res, array(mps[i], links[i-1], sites[i], links[i]))
        end
    end
    return res
end

function calc_itpl_point_errs(itpl_point_dict, fdata)
    res = 0
    for (key, value) in itpl_point_dict
        idx = Q.quantics_to_index(key)[1]
        res += abs(fdata[idx] - value)
    end
    return res
end


calc_itpl_point_errs (generic function with 1 method)

In [9]:
# 各資産ともグリッドの最大値を使う場合
localindex_v = [100 for _ in 1:2d]

mean_pv, std_pv = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
println("Present Value: ", mean_pv)
println("Standard Deviation: ", std_pv)
println("Abs.Err.: ", std_pv / sqrt(nPath))


Present Value: 6.16224477441282
Standard Deviation: 9.312806162122191
Abs.Err.: 2.944967887995743


In [10]:
using Random

d = 5
num_grid = 100

# 誤差を記録する配列
abs_errors = Float64[]

Random.seed!(seed)
for _ in 1:100
    # [σ1, S0_1, σ2, S0_2, ..., σd, S0_d] をランダムに生成
    localindex_v = [rand(1:num_grid) for _ in 1:2d]
    mean_pv, std_pv = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
    abs_err = std_pv / sqrt(nPath)
    push!(abs_errors, abs_err)
end

println("Mean Abs.Err.: ", mean(abs_errors))
println("Max  Abs.Err.: ", maximum(abs_errors))


Mean Abs.Err.: 1.0704463332803593
Max  Abs.Err.: 2.680704658749252


In [18]:
abo_(localindex_v) = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)[1]
localdims_v = fill(num_nodes, 2d)
Random.seed!(1234)
firstpivot_v = rand(1:num_nodes, 2d)
abo = WrapFuncFloat(abo_) # 
phi = TCI.CachedFunction{Float64}(abo, localdims_v)

phi = TCI.CachedFunction{Float64}(abo, localdims_v)




(::TensorCrossInterpolation.CachedFunction{Float64, UInt128}) (generic function with 3 methods)

In [19]:
# TCIワンショット補間関数
function tci_oneshot(func, d, localdims, firstpivot, tol)
    BLAS.set_num_threads(4)
    for _ in 1:100
        p = TCI.optfirstpivot(func, localdims, firstpivot)
        if abs(func(p)) > abs(func(firstpivot))
            firstpivot = p
        end
    end
    qtt, ranks, errors = TCI.crossinterpolate2(Float64, func, localdims, [firstpivot],
                                                  tolerance=tol, maxiter=1, verbosity=1,
                                                  loginterval=1, pivotsearch=:rook, maxbonddim=1)
    return qtt, errors
end

tci_oneshot (generic function with 1 method)

In [20]:
tol_mc = 2.0
tol = tol_mc

tci_time = @elapsed begin
    tt_tci, errors_asianbarrier = tci_oneshot(phi, d, localdims_v, firstpivot_v, tol)
end

iteration = 1, rank = 1, error= 0.0, maxsamplevalue= 9.316950965038105, nglobalpivot=0


152.138707792

In [21]:
tt_tci ###

(::TensorCrossInterpolation.TensorCI2{Float64}) (generic function with 1 method)

In [22]:
tt0 = TCI.TensorTrain(tt_tci)
itpl_point_dict = abo.itpl_point_dict
indexsets = sort(collect(keys(itpl_point_dict)))
vals = [itpl_point_dict[i] for i in indexsets]
fit_itr = 100 #
ttfit = TCI.TensorTrainFit{Float64}(indexsets, vals, tt0)
x0::Vector{Float64} = TCI.flatten(tt0)

using Zygote
using OptimKit
#using OptimKit
function loss(x) 
    ttfit(x)
end
optimizer = LBFGS(; maxiter=fit_itr, verbosity=1)
function loss_and_grad(x)
    y, (∇,) = withgradient(loss, x)
    return y, ∇
end
xopt, fs, gs, niter, normgradhistory = optimize(loss_and_grad, x0, optimizer)
;

ttopt = TCI.TensorTrain{Float64, 3}(TCI.to_tensors(ttfit, xopt))
tt_tci = ttopt

┌ Warning: LBFGS: not converged to requested tol after 100 iterations and time 1667.89 s: f = 4720.262510481811, ‖∇f‖ = 2.0325e+03
└ @ OptimKit /Users/sakurairihito/.julia/packages/OptimKit/G6i79/src/lbfgs.jl:197


(::TensorTrain{Float64, 3}) (generic function with 1 method)

In [ ]:
JLD2.@save "tt_tci_opt_$nPath" tt_tci

UndefVarError: UndefVarError: `tt_tci` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [16]:
JLD2.@load "tt_tci_$nPath" tt_tci

KeyError: KeyError: key "tt_tci" not found